# Base

In [1]:
%%capture
!apt-get install maven -qq

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import os
import json
import glob
import sys

In [3]:
%%capture
!git clone --recurse-submodules https://github.com/castorini/anserini.git
%cd anserini
!cd tools/eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../../..
!mvn clean package appassembler:assemble -DskipTests -Dmaven.javadoc.skip=true

In [4]:
#import the necessary data (i am not sure this is the latest data)
!mkdir collections/fever
!mkdir indexes/fever

!wget https://s3-eu-west-1.amazonaws.com/fever.public/feverous/feverous-wiki-pages.zip -P collections/fever
!unzip collections/fever/feverous-wiki-pages.zip -d collections/fever

!wget https://s3-eu-west-1.amazonaws.com/fever.public/feverous/train.jsonl -P collections/fever
!wget https://s3-eu-west-1.amazonaws.com/fever.public/feverous/dev.jsonl -P collections/fever

--2021-07-15 08:10:46--  https://s3-eu-west-1.amazonaws.com/fever.public/feverous/feverous-wiki-pages.zip
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.41.43
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.41.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9906569155 (9.2G) [application/zip]
Saving to: ‘collections/fever/feverous-wiki-pages.zip’

feverous-wiki-pages 100%[===================>]   9.23G  11.6MB/s    in 13m 38s 

2021-07-15 08:24:25 (11.6 MB/s) - ‘collections/fever/feverous-wiki-pages.zip’ saved [9906569155/9906569155]

Archive:  collections/fever/feverous-wiki-pages.zip
   creating: collections/fever/FeverousWikiv1/
  inflating: collections/fever/FeverousWikiv1/wiki_283.jsonl  
  inflating: collections/fever/FeverousWikiv1/wiki_402.jsonl  
  inflating: collections/fever/FeverousWikiv1/wiki_177.jsonl  
  inflating: collections/fever/FeverousWikiv1/wiki_287.jsonl  
  inflating: collectio

In [5]:
!ls collections/fever/FeverousWikiv1

wiki_000.jsonl	wiki_109.jsonl	wiki_218.jsonl	wiki_327.jsonl	wiki_436.jsonl
wiki_001.jsonl	wiki_110.jsonl	wiki_219.jsonl	wiki_328.jsonl	wiki_437.jsonl
wiki_002.jsonl	wiki_111.jsonl	wiki_220.jsonl	wiki_329.jsonl	wiki_438.jsonl
wiki_003.jsonl	wiki_112.jsonl	wiki_221.jsonl	wiki_330.jsonl	wiki_439.jsonl
wiki_004.jsonl	wiki_113.jsonl	wiki_222.jsonl	wiki_331.jsonl	wiki_440.jsonl
wiki_005.jsonl	wiki_114.jsonl	wiki_223.jsonl	wiki_332.jsonl	wiki_441.jsonl
wiki_006.jsonl	wiki_115.jsonl	wiki_224.jsonl	wiki_333.jsonl	wiki_442.jsonl
wiki_007.jsonl	wiki_116.jsonl	wiki_225.jsonl	wiki_334.jsonl	wiki_443.jsonl
wiki_008.jsonl	wiki_117.jsonl	wiki_226.jsonl	wiki_335.jsonl	wiki_444.jsonl
wiki_009.jsonl	wiki_118.jsonl	wiki_227.jsonl	wiki_336.jsonl	wiki_445.jsonl
wiki_010.jsonl	wiki_119.jsonl	wiki_228.jsonl	wiki_337.jsonl	wiki_446.jsonl
wiki_011.jsonl	wiki_120.jsonl	wiki_229.jsonl	wiki_338.jsonl	wiki_447.jsonl
wiki_012.jsonl	wiki_121.jsonl	wiki_230.jsonl	wiki_339.jsonl	wiki_448.jsonl
wiki_013.jsonl	wiki_122.j

In [6]:
# A folder path, a file path, or a list of file paths is needed

# DATA_LOCATION = "wiki_pages"
# DATA_LOCATION = "wiki_pages/wiki_000.jsonl"
DATA_LOCATION = [
    "collections/fever/FeverousWikiv1/wiki_000.jsonl",
    "collections/fever/FeverousWikiv1/wiki_001.jsonl",
    "collections/fever/FeverousWikiv1/wiki_002.jsonl",
    "collections/fever/FeverousWikiv1/wiki_003.jsonl",
    "collections/fever/FeverousWikiv1/wiki_004.jsonl",
]
OUTPUT_FOLDER = "wiki_pages_anserini"

SIMPLIFY_PAGE_LINKS = False
PREFER_URL_FOR_LINKS = False

# If SIMPLIFY_PAGE_LINKS and not PREFER_URL_FOR_LINKS: [[Page_(disambiguation)|Page Title]] -> Page Title
# If SIMPLIFY_PAGE_LINKS and PREFER_URL_FOR_LINKS: [[Page_(disambiguation)|Page Title]] -> Page (disambiguation)
# Note that the URL basename can sometimes contain extra information, but it can also redirect to a page that is different than the visible label.

SUBSECTION_SEP = "|"
SUBSUBSECTION_SEP = " "
import glob
import os

to_process = None

if type(DATA_LOCATION) == list:  # A list of file paths
    to_process = [file for file in DATA_LOCATION]
elif os.path.isfile(DATA_LOCATION):  # A file path
    to_process = [DATA_LOCATION]
elif os.path.isdir(DATA_LOCATION):  # A folder
    to_process = glob.glob("{}/*.jsonl".format(DATA_LOCATION))
else:
    raise ValueError("Data location is not a valid file or folder.")

print("{} file(s) will be processed.".format(len(to_process)))

import json
import re


def clean_value(value):
    """Replaces unnecessary space (including \t and such) with a single space
    character.

    Arg:
        value: A string.

    Returns:
        The simplified version of the provided string.
    """
    return " ".join(value.split())


def final_clean(
    field,
    simplify_page_links=SIMPLIFY_PAGE_LINKS,
    prefer_url_for_links=PREFER_URL_FOR_LINKS,
):
    """Removes multiple space characters with a single space character. Removes
    leading and trailing spaces. Simplifies the page links according to the
    parameters.

    Args:
        field (str): A field of the processed page data.
        simplify_page_links (Boolean): A Boolean indicating whether the page
            links must be simplified. By default, it is set to
            SIMPLIFY_PAGE_LINKS.
        prefer_url_for_links (Boolean): A Boolean indicating whether the page
            links must be simplified to the URL basename instead of the link's
            label. By default, it is set to SIMPLIFY_PAGE_LINKS.

    Returns:
        str: The simplified version of the provided string.
    """
    field = re.sub(" +", " ", field.strip(), flags=re.MULTILINE)
    if SIMPLIFY_PAGE_LINKS and PREFER_URL_FOR_LINKS:
        # Simplifies links by replacing them with the URL basename (replaces underscores)
        # Underscores are replaced with space
        field = re.sub(r"(?:\[\[)(.*)?(?:\|)", field.replace("_", " "), field)
        field = re.sub(r"(\|).*?(\]\])|\[\[", "", field, flags=re.MULTILINE)
    elif SIMPLIFY_PAGE_LINKS:
        # Simplifies links by replacing them with the titles
        field = re.sub(r"(\[\[).*?(\|)|]]", "", field, flags=re.MULTILINE)
    return field


if not os.path.exists(OUTPUT_FOLDER):  # Creates the output folder if it does not exist
    os.mkdir(OUTPUT_FOLDER)

counter = 0
for file_path in to_process:
    file_basename = os.path.basename(file_path)
    output_path = os.path.join(OUTPUT_FOLDER, file_basename)

    open(output_path, mode="w").close()  # Creating an empty output file

    with open(file_path, "r", encoding="utf-8") as f:
        file_data = [json.loads(line) for line in f]

    for page in file_data:

        page_data_processed = {"id": page["title"], "text": "", "lines": ""}

        # The order list of the page will be used to parse the items
        for item_id, item in enumerate(page["order"]):
            if item.startswith("sentence_"):  # Sentence
                clean_element = clean_value(page[item])
                page_data_processed["text"] += " {}".format(clean_element)
                page_data_processed["lines"] += "{}\t{}\n".format(item, clean_element)
            elif item.startswith("table_"):  # Table
                page_data_processed["text"] += " "
                page_data_processed["lines"] += "{}\t".format(item)
                table_rows = []
                for row in page[item]["table"]:
                    row_items = []
                    for cell in row:
                        row_items.append(clean_value(cell["value"]))

                    row_text = "{}".format(SUBSUBSECTION_SEP).join(row_items)
                    table_rows.append(row_text)

                if "caption" in page[item]:
                    table_rows.append(clean_value(page[item]["caption"]))

                table_text = "{}".format(SUBSECTION_SEP).join(table_rows)
                page_data_processed["text"] += table_text
                page_data_processed["lines"] += table_text
                page_data_processed["lines"] += "\n"
            elif item.startswith("section_"):  # Section
                clean_element = clean_value(page[item]["value"])
                page_data_processed["text"] += " {}".format(clean_element)
                page_data_processed["lines"] += "{}\t{}\n".format(item, clean_element)
            elif item.startswith("list_"):  # List
                list_items = []
                for list_item in page[item]["list"]:
                    list_items.append(clean_value(list_item["value"]))

                list_text = "{}".format(SUBSECTION_SEP).join(list_items)
                page_data_processed["text"] += list_text
                page_data_processed["lines"] += "{}\t{}\n".format(item, list_text)
            else:  # All alternatives must be handled and the code must not reach here
                raise ValueError("Unidentified page element found.")

        page_data_processed["text"] = final_clean(page_data_processed["text"])
        page_data_processed["lines"] = final_clean(page_data_processed["lines"])

        # Appending the processed page data to the file
        with open(output_path, "a", encoding="utf-8") as f:
            json.dump(page_data_processed, f, ensure_ascii=False)
            f.write("\n")

5 file(s) will be processed.


In [7]:
!sh target/appassembler/bin/IndexCollection -collection FeverParagraphCollection \
 -input wiki_pages_anserini \
 -index indexes/fever/lucene-index-fever-paragraph \
 -generator DefaultLuceneDocumentGenerator \
 -threads 1 -storePositions -storeDocvectors -storeRaw \
  >& logs/log.fever &

In [8]:
!tar -cvf "/content/anserini/indexes/fever/lucene-index-fever-paragraph.tar" "/content/anserini/indexes/fever/lucene-index-fever-paragraph" #Tar operation for exporting the index and make it reusable
# Only one indexing will be enough we will use the tar file after that use the code below to untar it and equate to the LucuneDir
#!tar xvfz lucene-index-fever-paragraph.tar

tar: Removing leading `/' from member names
tar: /content/anserini/indexes/fever/lucene-index-fever-paragraph: Cannot stat: No such file or directory
tar: Exiting with failure status due to previous errors


In [9]:
luceneDir = 'indexes/fever/lucene-index-fever-paragraph'

In [10]:
USE_SUMMARY = False

In [11]:
import os
#%%capture
!curl -O https://download.java.net/java/GA/jdk11/9/GPL/openjdk-11.0.2_linux-x64_bin.tar.gz
!mv openjdk-11.0.2_linux-x64_bin.tar.gz /usr/lib/jvm/; cd /usr/lib/jvm/; tar -zxvf openjdk-11.0.2_linux-x64_bin.tar.gz
!update-alternatives --install /usr/bin/java java /usr/lib/jvm/jdk-11.0.2/bin/java 1
!update-alternatives --set java /usr/lib/jvm/jdk-11.0.2/bin/java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/jdk-11.0.2"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  178M  100  178M    0     0   108M      0  0:00:01  0:00:01 --:--:--  108M
jdk-11.0.2/bin/jaotc
jdk-11.0.2/bin/jar
jdk-11.0.2/bin/jarsigner
jdk-11.0.2/bin/java
jdk-11.0.2/bin/javac
jdk-11.0.2/bin/javadoc
jdk-11.0.2/bin/javap
jdk-11.0.2/bin/jcmd
jdk-11.0.2/bin/jconsole
jdk-11.0.2/bin/jdb
jdk-11.0.2/bin/jdeprscan
jdk-11.0.2/bin/jdeps
jdk-11.0.2/bin/jhsdb
jdk-11.0.2/bin/jimage
jdk-11.0.2/bin/jinfo
jdk-11.0.2/bin/jjs
jdk-11.0.2/bin/jlink
jdk-11.0.2/bin/jmap
jdk-11.0.2/bin/jmod
jdk-11.0.2/bin/jps
jdk-11.0.2/bin/jrunscript
jdk-11.0.2/bin/jshell
jdk-11.0.2/bin/jstack
jdk-11.0.2/bin/jstat
jdk-11.0.2/bin/jstatd
jdk-11.0.2/bin/keytool
jdk-11.0.2/bin/pack200
jdk-11.0.2/bin/rmic
jdk-11.0.2/bin/rmid
jdk-11.0.2/bin/rmiregistry
jdk-11.0.2/bin/serialver
jdk-11.0.2/bin/unpack200
jdk-11.0.2/conf/logging.properties
jdk-11.0.2/conf/management/jmx

In [12]:
import tensorflow as tf
import tensorflow_hub as hub
!mkdir ~/kaggle
!mkdir ~/kaggle/working/
!mkdir ~/kaggle/working/sentence_wise_email/
!mkdir ~/kaggle/working/sentence_wise_email/module/
!mkdir ~/kaggle/working/sentence_wise_email/module/module_useT
# Download the module, and uncompress it to the destination folder. 
!curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/3?tf-hub-format=compressed" | tar -zxvC ~/kaggle/working/sentence_wise_email/module/module_useT

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0./
./tfhub_module.pb
./variables/
./variables/variables.data-00000-of-00001
 95  745M   95  711M    0     0  29.7M      0  0:00:25  0:00:23  0:00:02 46.3M./variables/variables.index
./assets/
./saved_model.pb
100  745M  100  745M    0     0  30.2M      0  0:00:24  0:00:24 --:--:-- 46.6M


In [13]:
import torch
!pip install transformers
!pip install --no-cache-dir transformers sentencepiece
#from transformers import BertForQuestionAnswering
#from transformers import BertTokenizer
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

torch_device = 'cuda'

#dmis-lab/biobert-base-cased-v1.1-squad
#abhijithneilabraham/longformer_covid_qa
#graviraja/covidbert_squad 

QA_TOKENIZER = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1-squad")
QA_MODEL = AutoModelForQuestionAnswering.from_pretrained("dmis-lab/biobert-base-cased-v1.1-squad")

#QA_MODEL = AutoModelForQuestionAnswering.from_pretrained("NeuML/bert-small-cord19qa")
#QA_TOKENIZER = AutoTokenizer.from_pretrained("NeuML/bert-small-cord19qa")

#QA_MODEL = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
#QA_TOKENIZER = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
QA_MODEL.to(torch_device)
QA_MODEL.eval()
#Auto Summarization with BART
if USE_SUMMARY:
    SUMMARY_TOKENIZER = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    SUMMARY_MODEL = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

    SUMMARY_MODEL.to(torch_device)
    SUMMARY_MODEL.eval()

     |████████████████████████████████| 2.5MB 5.1MB/s 
     |████████████████████████████████| 3.3MB 16.2MB/s 
     |████████████████████████████████| 901kB 34.5MB/s 
     |████████████████████████████████| 1.2MB 4.6MB/s 


In [14]:
#%%capture
!pip install pyserini==0.8.1.0
from pyserini.search import pysearch

     |████████████████████████████████| 57.7MB 98kB/s 
     |████████████████████████████████| 1.1MB 6.9MB/s 


In [15]:
query_processed = 'As we have mentioned above, COVID-19 is only fatal for a small percentage of people. In fact 80% face only mild symptoms with no need for hospitalization?'
query = 'As we have mentioned above, COVID-19 is only fatal for a small percentage of people. In fact 80% face only mild symptoms with no need for hospitalization?'

keywords = '2019-nCoV, SARS-CoV-2, COVID-19, symptoms, hospitalization'

In [16]:
import json

searcher = pysearch.SimpleSearcher(luceneDir)
hits = searcher.search(query_processed + '. ' + keywords,k=50) # We can retrieve the documents according to concatanated string of query and keywords we can change this setup.
n_hits = len(hits)
## collect the relevant data in a hit dictionary
hit_dictionary = {}

In [17]:
import json
#'As we have mentioned above, COVID-19 is only fatal for a small percentage of people. In fact 80% face only mild symptoms with no need for hospitalization?'

searcher = pysearch.SimpleSearcher(luceneDir)
hits = searcher.search(query_processed + '. ' + keywords,k=50)
n_hits = len(hits)
## collect the relevant data in a hit dictionary
hit_dictionary = {}
for i in range(0, n_hits):
  if 1==1:
    doc_json=dict()
    idx = str(hits[i].docid)
    hit_dictionary[idx] = doc_json
    hit_dictionary[idx]['abstract']=hits[i].lucene_document.get("raw")
    hit_dictionary[idx]['real_abstract']=hits[i].lucene_document.get("raw")
    hit_dictionary[idx]['title'] = str(hits[i].docid)

## scrub the abstracts in prep for BERT-SQuAD
for idx,v in hit_dictionary.items():
    abs_dirty = v['abstract']
    real_abs_dirty = v['real_abstract']
    #abs_dirty = v['paragraph']
    # looks like the abstract value can be an empty list
    v['abstract_paragraphs'] = []
    v['abstract_full'] = ''
    v['real_abstract_full'] = ''
    v['real_abstract_paragraphs']=[]

    if abs_dirty:
        # looks like if it is a list, then the only entry is a dictionary wher text is in 'text' key
        # looks like it is broken up by paragraph if it is in that form.  lets make lists for every paragraph
        # and a new entry that is full abstract text as both could be valuable for BERT derrived QA


        if isinstance(abs_dirty, list):
            for p in abs_dirty:
                v['abstract_paragraphs'].append(p['text'])
                v['abstract_full'] += p['text'] + ' \n\n'

        # looks like in some cases the abstract can be straight up text so we can actually leave that alone
        if isinstance(abs_dirty, str):
            v['abstract_paragraphs'].append(abs_dirty)
            v['abstract_full'] += abs_dirty + ' \n\n'
    if real_abs_dirty:
        # looks like if it is a list, then the only entry is a dictionary wher text is in 'text' key
        # looks like it is broken up by paragraph if it is in that form.  lets make lists for every paragraph
        # and a new entry that is full abstract text as both could be valuable for BERT derrived QA


        if isinstance(real_abs_dirty, list):
            for p in real_abs_dirty:
                v['real_abstract_paragraphs'].append(p['text'])
                v['real_abstract_full'] += p['text'] + ' \n\n'

        # looks like in some cases the abstract can be straight up text so we can actually leave that alone
        if isinstance(abs_dirty, str):
            v['real_abstract_paragraphs'].append(real_abs_dirty)
            v['real_abstract_full'] += real_abs_dirty + ' \n\n'
    v["indexed_para"]=v["abstract_full"][len(v["real_abstract_full"]):]

In [18]:
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.compat.v1.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.compat.v1.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})
embed_fn = embed_useT('/root/kaggle/working/sentence_wise_email/module/module_useT')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [19]:
import numpy as np
#Reconstruct text for generating the answer text from the BERT result, do the necessary text preprocessing and construct the answer.
def reconstructText(tokens, start=0, stop=-1):
    tokens = tokens[start: stop]
    if '[SEP]' in tokens:
        sepind = tokens.index('[SEP]')
        tokens = tokens[sepind+1:]
    txt = ' '.join(tokens)
    txt = txt.replace(' ##', '')
    txt = txt.replace('##', '')
    txt = txt.strip()
    txt = " ".join(txt.split())
    txt = txt.replace(' .', '.')
    txt = txt.replace('( ', '(')
    txt = txt.replace(' )', ')')
    txt = txt.replace(' - ', '-')
    txt_list = txt.split(' , ')
    txt = ''
    nTxtL = len(txt_list)
    if nTxtL == 1:
        return txt_list[0]
    newList =[]
    for i,t in enumerate(txt_list):
        if i < nTxtL -1:
            if t[-1].isdigit() and txt_list[i+1][0].isdigit():
                newList += [t,',']
            else:
                newList += [t, ', ']
        else:
            newList += [t]
    return ''.join(newList)


def makeBERTSQuADPrediction(document, question):
    ## we need to rewrite this function so that it chuncks the document into 250-300 word segments with
    ## 50 word overlaps on either end so that it can understand and check longer abstracts
    ## Get chuck of documents in order to overcome the BERT's max 512 token problem. First count the number of tokens. For example if it is
    ##   larger than 2048 token determine the split size and overlapping token count. Split the document into pieces to fed into BERT encoding. Overlapping
    ##      is necessary to maintain the coherence between the splits. If the token count is larger than some number 2560 token approximately this will fail.
    nWords = len(document.split())
    input_ids_all = QA_TOKENIZER.encode(question, document)
    tokens_all = QA_TOKENIZER.convert_ids_to_tokens(input_ids_all)
    overlapFac = 1.1
    if len(input_ids_all)*overlapFac > 2560:
        nSearchWords = int(np.ceil(nWords/6))
        fifth = int(np.ceil(nWords/5))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                     ' '.join(docSplit[fifth-int(nSearchWords*overlapFac/2):fifth+int(fifth*overlapFac/2)]),
                     ' '.join(docSplit[fifth*2-int(nSearchWords*overlapFac/2):fifth*2+int(fifth*overlapFac/2)]),
                     ' '.join(docSplit[fifth*3-int(nSearchWords*overlapFac/2):fifth*3+int(fifth*overlapFac/2)]),
                     ' '.join(docSplit[fifth*4-int(nSearchWords*overlapFac/2):fifth*4+int(fifth*overlapFac/2)]),
                     ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]

    elif len(input_ids_all)*overlapFac > 2048:
        nSearchWords = int(np.ceil(nWords/5))
        quarter = int(np.ceil(nWords/4))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                     ' '.join(docSplit[quarter-int(nSearchWords*overlapFac/2):quarter+int(quarter*overlapFac/2)]),
                     ' '.join(docSplit[quarter*2-int(nSearchWords*overlapFac/2):quarter*2+int(quarter*overlapFac/2)]),
                     ' '.join(docSplit[quarter*3-int(nSearchWords*overlapFac/2):quarter*3+int(quarter*overlapFac/2)]),
                     ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]        
        
    elif len(input_ids_all)*overlapFac > 1536:
        nSearchWords = int(np.ceil(nWords/4))
        third = int(np.ceil(nWords/3))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                     ' '.join(docSplit[third-int(nSearchWords*overlapFac/2):third+int(nSearchWords*overlapFac/2)]),
                     ' '.join(docSplit[third*2-int(nSearchWords*overlapFac/2):third*2+int(nSearchWords*overlapFac/2)]),
                     ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]        
        
    elif len(input_ids_all)*overlapFac > 1024:
        nSearchWords = int(np.ceil(nWords/3))
        middle = int(np.ceil(nWords/2))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                     ' '.join(docSplit[middle-int(nSearchWords*overlapFac/2):middle+int(nSearchWords*overlapFac/2)]),
                     ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
    elif len(input_ids_all)*overlapFac > 512:
        nSearchWords = int(np.ceil(nWords/2))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
    else:
        input_ids = [input_ids_all]
    absTooLong = False    
    
    answers = []
    cons = []
    #print(input_ids)
    for iptIds in input_ids:
        tokens = QA_TOKENIZER.convert_ids_to_tokens(iptIds)
        #print(tokens)
        sep_index = iptIds.index(QA_TOKENIZER.sep_token_id)
        num_seg_a = sep_index + 1
        num_seg_b = len(iptIds) - num_seg_a
        segment_ids = [0]*num_seg_a + [1]*num_seg_b
        assert len(segment_ids) == len(iptIds)
        n_ids = len(segment_ids)
        #print(n_ids)
        if n_ids < 512:
            outputs = QA_MODEL(torch.tensor([iptIds]).to(torch_device), 
                                     token_type_ids=torch.tensor([segment_ids]).to(torch_device))
            #print(QA_MODEL(torch.tensor([iptIds]).to(torch_device), 
             #                        token_type_ids=torch.tensor([segment_ids]).to(torch_device)))
        else:
            #this cuts off the text if its more than 512 words so it fits in model space
            #need run multiple inferences for longer text. add to the todo. I will try to expand this to longer sequences if it is necessary. 
            print('****** warning only considering first 512 tokens, document is '+str(nWords)+' words long.  There are '+str(n_ids)+ ' tokens')
            absTooLong = True
            outputs= QA_MODEL(torch.tensor([iptIds[:512]]).to(torch_device), 
                                     token_type_ids=torch.tensor([segment_ids[:512]]).to(torch_device))
        start_scores=outputs.start_logits
        end_scores=outputs.end_logits
        start_scores = start_scores[:,1:-1]
        end_scores = end_scores[:,1:-1]
        answer_start = torch.argmax(start_scores)
        answer_end = torch.argmax(end_scores)
        #print(answer_start, answer_end)
        answer = reconstructText(tokens, answer_start, answer_end+2)
    
        if answer.startswith('. ') or answer.startswith(', '):
            answer = answer[2:]
            
        c = start_scores[0,answer_start].item()+end_scores[0,answer_end].item()
        answers.append(answer)
        cons.append(c)
    
    maxC = max(cons)
    iMaxC = [i for i, j in enumerate(cons) if j == maxC][0]
    confidence = cons[iMaxC]
    answer = answers[iMaxC]
    
    sep_index = tokens_all.index('[SEP]')
    full_txt_tokens = tokens_all[sep_index+1:]
    
    abs_returned = reconstructText(full_txt_tokens)

    ans={}
    ans['answer'] = answer
    #print(answer)
    if answer.startswith('[CLS]') or answer_end.item() < sep_index or answer.endswith('[SEP]'):
        ans['confidence'] = -1000000
    else:
        #confidence = torch.max(start_scores) + torch.max(end_scores)
        #confidence = np.log(confidence.item())
        ans['confidence'] = confidence
    #ans['start'] = answer_start.item()
    #ans['end'] = answer_end.item()
    ans['abstract_bert'] = abs_returned
    ans['abs_too_long'] = absTooLong
    return ans

In [20]:
from tqdm import tqdm
def searchAbstracts(hit_dictionary, question):
    abstractResults = {}
    for k,v in tqdm(hit_dictionary.items()):
        abstract = v['abstract_full']
        indexed_para=v['indexed_para']
        if abstract:
            ans = makeBERTSQuADPrediction(abstract, question)
            if ans['answer']:
                confidence = ans['confidence']
                abstractResults[confidence]={}
                abstractResults[confidence]['answer'] = ans['answer']
                #abstractResults[confidence]['start'] = ans['start']
                #abstractResults[confidence]['end'] = ans['end']
                abstractResults[confidence]['abstract_bert'] = ans['abstract_bert']
                abstractResults[confidence]['idx'] = k
                abstractResults[confidence]['abs_too_long'] = ans['abs_too_long']

                
    cList = list(abstractResults.keys())

    if cList:
        maxScore = max(cList)
        total = 0.0
        exp_scores = []
        for c in cList:
            s = np.exp(c-maxScore)
            exp_scores.append(s)
        total = sum(exp_scores)
        for i,c in enumerate(cList):
            abstractResults[exp_scores[i]/total] = abstractResults.pop(c)
    return abstractResults

In [21]:
answers = searchAbstracts(hit_dictionary, query)

  0%|          | 0/50 [00:00<?, ?it/s]

****** warning only considering first 512 tokens, document is 3699 words long.  There are 1526 tokens
****** warning only considering first 512 tokens, document is 3699 words long.  There are 1706 tokens
****** warning only considering first 512 tokens, document is 3699 words long.  There are 1361 tokens


  2%|▏         | 1/50 [00:00<00:22,  2.17it/s]

****** warning only considering first 512 tokens, document is 3699 words long.  There are 1641 tokens
****** warning only considering first 512 tokens, document is 3699 words long.  There are 1750 tokens
****** warning only considering first 512 tokens, document is 3699 words long.  There are 1904 tokens
****** warning only considering first 512 tokens, document is 2309 words long.  There are 685 tokens
****** warning only considering first 512 tokens, document is 2309 words long.  There are 743 tokens


  4%|▍         | 2/50 [00:00<00:18,  2.56it/s]

...

****** warning only considering first 512 tokens, document is 1209 words long.  There are 516 tokens
****** warning only considering first 512 tokens, document is 1209 words long.  There are 644 tokens
****** warning only considering first 512 tokens, document is 1209 words long.  There are 569 tokens
****** warning only considering first 512 tokens, document is 2474 words long.  There are 941 tokens
****** warning only considering first 512 tokens, document is 2474 words long.  There are 1169 tokens
****** warning only considering first 512 tokens, document is 2474 words long.  There are 1183 tokens
****** warning only considering first 512 tokens, document is 2474 words long.  There are 901 tokens
****** warning only considering first 512 tokens, document is 2474 words long.  There are 888 tokens
****** warning only considering first 512 tokens, document is 2474 words long.  There are 1235 tokens


 98%|█████████▊| 49/50 [00:19<00:00,  3.53it/s]

****** warning only considering first 512 tokens, document is 6000 words long.  There are 2560 tokens
****** warning only considering first 512 tokens, document is 6000 words long.  There are 2210 tokens
****** warning only considering first 512 tokens, document is 6000 words long.  There are 2449 tokens
****** warning only considering first 512 tokens, document is 6000 words long.  There are 2243 tokens
****** warning only considering first 512 tokens, document is 6000 words long.  There are 2718 tokens


100%|██████████| 50/50 [00:20<00:00,  2.49it/s]

****** warning only considering first 512 tokens, document is 6000 words long.  There are 2359 tokens


In [22]:
FIND_PDFS=False 
SEARCH_MEDRXIV=False

In [23]:
workingPath = '/root/kaggle/working'
import pandas as pd
import re
if FIND_PDFS:
    from metapub import UrlReverse
    from metapub import FindIt
from IPython.core.display import display, HTML

#from summarizer import Summarizer
#summarizerModel = Summarizer()
def displayResults(hit_dictionary, answers, question):
    
    question_HTML = '<div style="font-family: Times New Roman; font-size: 28px; padding-bottom:28px"><b>Query</b>: '+question+'</div>'
    #all_HTML_txt = question_HTML
    confidence = list(answers.keys())
    confidence.sort(reverse=True)
    
    confidence = list(answers.keys())
    confidence.sort(reverse=True)
    

    for c in confidence:
        if c>0 and c <= 1 and len(answers[c]['answer']) != 0:
            if 'idx' not in  answers[c]:
                continue
            rowData = []
            idx = answers[c]['idx']
            title = hit_dictionary[idx]['title']

            
            full_abs = answers[c]['abstract_bert']
            bert_ans = answers[c]['answer']
            
            
            #split_abs = full_abs.split(bert_ans)
            #x=(split_abs[0][:split_abs[0].rfind('. ')+1])
            #sentance_beginning = x[x.rfind('. ')+1:] + split_abs[0][split_abs[0].rfind('. ')+1:]
            #sentance_beginning = split_abs[0][split_abs[0].rfind('.')+1:]
            x=''
            y=''
            z=''
            t=''
            regex = r"(\.\s[^0-9])(?!.*(\.\s[^0-9]))"
            split_abs = full_abs.split(bert_ans)
            matches = re.finditer(regex, split_abs[0], re.MULTILINE)
            for matchNum1, match in enumerate(matches, start=1):
              y=(split_abs[0][:match.start()+1])
            matches = re.finditer(regex, y, re.MULTILINE)
            for matchNum2, match in enumerate(matches, start=1):
              x=(y[match.start()+1:])
            if x=='':
              x=split_abs[0]
            matches = re.finditer(regex, split_abs[0], re.MULTILINE)
            for matchNum3, match in enumerate(matches, start=1):
              z=(split_abs[0][match.start()+1:])  
            sentance_beginning = x+z
            regex2=r"(.*?(?<!\b\w)[.?!])\s+[a-zA-Z0-9]"
            if len(split_abs) == 1:
                sentance_end_pos = len(full_abs)
                sentance_end =''
            else:
                matches = re.finditer(regex2, split_abs[1], re.MULTILINE)
                for matchNum4, match in enumerate(matches, start=1):
                    if matchNum4==1:
                        t=(split_abs[1][:match.end()-2])
                sentance_end_pos = split_abs[1].find('. ')+1
                if sentance_end_pos == 0:
                    sentance_end = split_abs[1]
                else:
                    sentance_end = t
            #if len(split_abs) == 1:
            #    sentance_end_pos = len(full_abs)
            #    sentance_end =''
            #else:
            #    sentance_end_pos = split_abs[1].find('. ')+1
            #    if sentance_end_pos == 0:
            #      sentance_end = split_abs[1]
            #    else:
            #      sentance_end = split_abs[1][:sentance_end_pos]
                
            #sentance_full = sentance_beginning + bert_ans+ sentance_end
            answers[c]['full_answer'] = sentance_beginning+bert_ans+sentance_end
            answers[c]['partial_answer'] = bert_ans+sentance_end
            answers[c]['sentence_beginning'] = sentance_beginning
            answers[c]['sentence_end'] = sentance_end
            answers[c]['title'] = title
        else:
            answers.pop(c)
    
    
    ## now rerank based on semantic similarity of the answers to the question
    ## Universal sentence encoder
    cList = list(answers.keys())
    allAnswers = [answers[c]['full_answer'] for c in cList]
    
    messages = [question]+allAnswers
    
    encoding_matrix = embed_fn(messages)
    similarity_matrix = np.inner(encoding_matrix, encoding_matrix)
    rankings = similarity_matrix[1:,0]
    
    for i,c in enumerate(cList):
        answers[rankings[i]] = answers.pop(c)

    ## now form pandas dv
    confidence = list(answers.keys())
    confidence.sort(reverse=True)
    pandasData = []
    ranked_aswers = []
    full_abs_list=[]
    for c in confidence:
        rowData=[]
        title = answers[c]['title']
        idx = answers[c]['idx']
        rowData += [idx]            
        sentance_html = '<div>' +answers[c]['sentence_beginning'] + " <font color='red'>"+answers[c]['answer']+"</font> "+answers[c]['sentence_end']+'</div>'
        
        rowData += [sentance_html, c,title]
        pandasData.append(rowData)
        ranked_aswers.append(' '.join([answers[c]['full_answer']]))
        full_abs_list.append(' '.join([answers[c]['abstract_bert']]))
    
    if FIND_PDFS or SEARCH_MEDRXIV:
        pdata2 = []
        pm_ids = []
        for rowData in pandasData:
            rd = rowData
            idx = rowData[0]
    else:
        pdata2 = pandasData
        
    
    display(HTML(question_HTML))

    if USE_SUMMARY:
        ## try generating an exacutive summary with bart abstractive summarizer
        allAnswersTxt = ' '.join(ranked_aswers[:10]).replace('\n','')
    #    exec_sum = summarizerModel(allAnswersTxt, min_length=1, max_length=500)    
     #   execSum_HTML = '<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:18px"><b>BERT Extractive Summary:</b>: '+exec_sum+'</div>'

        answers_input_ids = SUMMARY_TOKENIZER.batch_encode_plus([allAnswersTxt], return_tensors='pt', max_length=2048)['input_ids'].to(torch_device)
        #answers_input_ids = SUMMARY_TOKENIZER.prepare_seq2seq_batch([allAnswersTxt], return_tensors='pt', truncation=True, padding='longest')['input_ids'].to(torch_device)
        summary_ids = SUMMARY_MODEL.generate(answers_input_ids,
                                               num_beams=10,
                                               length_penalty=1.1,
                                               max_length=2048,
                                               min_length=64,
                                               no_repeat_ngram_size=0,
                                                do_sample=False )

        exec_sum = SUMMARY_TOKENIZER.decode(summary_ids.squeeze(), skip_special_tokens=True)
        execSum_HTML = '<div style="font-family: Times New Roman; font-size: 18px; margin-bottom:1pt"><b>BART Abstractive Summary:</b>: '+exec_sum+'</div>'
        display(HTML(execSum_HTML))
        warning_HTML = '<div style="font-family: Times New Roman; font-size: 12px; padding-bottom:12px; color:#CCCC00; margin-top:1pt"> Warning this is an autogenerated summary based on semantic search of abstracts, always examine the sources before accepting this conclusion.  If the evidence only mentions topic in passing or the evidence is not clear, the summary will likely not clearly answer the question.</div>'
        display(HTML(warning_HTML))

#    display(HTML('<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:18px"><b>Body of Evidence:</b></div>'))
    
    if FIND_PDFS or SEARCH_MEDRXIV:
        df = pd.DataFrame(pdata2, columns = ['Lucene ID', 'BERT-SQuAD Answer with Highlights', 'Confidence', 'Title/Link'])
    else:
        df = pd.DataFrame(pdata2, columns = ['Lucene ID', 'BERT-SQuAD Answer with Highlights', 'Confidence', 'Title/Link'])
        
    display(HTML(df.to_html(render_links=True, escape=False)))
    return full_abs_list,ranked_aswers,pandasData

    


In [24]:
full_abs_list,ranked_aswers,pm_ids=displayResults(hit_dictionary, answers, query)

,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link
0,Lassa fever,"sentence _ 90 as of 2003,10-16 % of people in sierra leone and liberia admitted to hospital had the virus. sentence _ 91 the case fatality rate for those who are hospitalized for the disease is about 15-20 % .",0.586256,Lassa fever
1,Acute disseminated encephalomyelitis,"sentence _ 11 adem shows seasonal variation with higher incidence in winter and spring months which may coincide with higher viral infections during these months. sentence _ 12 the mortality rate may be as high as 5 % ; however, full recovery is seen in 50 to 75 % of cases with increase in survival rates up to 70 to 90 % with figures including minor residual disability as well.",0.583253,Acute disseminated encephalomyelitis
2,Rosacea,"sentence _ 59 other cases, if left untreated, worsen over time. section _ 11 behavior sentence _ 60 avoiding triggers that worsen the condition can help reduce the onset of rosacea, but alone will not normally lead to remission except in mild cases .",0.578586,Rosacea
3,Transient global amnesia,"sentence _ 82 tga is most common in people between age 56 and 75, with the average age of a person experiencing tga being approximately 62. section _ 12 see also list _ 2 amnesia | [ [ dissociative _ amnesia | dissociative amnesia ] ] only mild symptoms with no need for hospitalization",0.573354,Transient global amnesia
4,Lung cancer,sentence _ 159 research has not found two other available tests — sputum cytology or [ [ chest _ radiograph | chest radiograph ] ] (cxr) screening tests — to have any benefit. sentence _ 160 the [ [ united _ states _ preventive _ services _ task _ force | united states preventive services task force ] ] (uspstf) recommends yearly screening using low-dose computed tomography in those who have a total smoking history of 30 pack-years and are between 55 and 80 years old until a person has not been smoking for more than 15 years.,0.550006,Lung cancer
5,Major trauma,"sentence _ 24 penetrating trauma is caused when a [ [ foreign _ body | foreign body ] ] such as a bullet or a knife enters the [ [ tissue _ (biology) | body tissue ] ], creating an open wound. sentence _ 25 in the united states, most deaths caused by penetrating trauma occur in urban areas and 80 % of these deaths are caused by firearms .",0.474236,Major trauma
6,Bucillamine,"food & drug administration (fda) has approved revive therapeutics ltd. to proceed with a randomized, double-blind, placebo-controlled confirmatory phase 3 clinical trial protocol to evaluate the safety and efficacy of bucillamine in patients with mild-moderate covid-19.",0.438401,Bucillamine
7,Arteriovenous malformation,"miller ] ] was diagnosed with avm after filming [ [ yogi _ bear _ (film) | yogi bear ] ] in new zealand in 2010 ; miller described his experience with the disease on the pete holmes podcast you made it weird on october 28,2011, shedding his comedian side for a moment and becoming more philosophical, narrating his behaviors and inability to sleep during that time. he suffered a seizure upon return to los angeles and successfully underwent surgery that had a mortality rate of ten percent . | jazz guitarist [ [ pat _ martino | pat martino ] ] experienced an avm and subsequently developed amnesia and manic depression.",0.400459,Arteriovenous malformation
8,Alcoholism,"sentence _ 2 for the song by starsailor, see [ [ alcoholic _ (song) | alcoholic (song) ] ]. table _ 0 alcoholism | other names alcohol dependence syndrome, alcohol use disorder (aud) | [ [ specialty _ (medicine) | specialty ] ] [ [ psychiatry | psychiatry ] ], [ [ clinical _ psychology | clinical psychology ] ], toxicology, addiction medicine | symptoms drinking large amounts of alcohol over a long period, difficulty cutting down, acquiring and drinking alcohol taking up a lot of time, usage resulting in problems, [ [ alcohol _ withdrawal _ syndrome | withdrawal ] ] occurring when stopping | [ [ complication _ (medicine) |

UnboundLocalError: ignored

In [ ]:
hits[0].lucene_document.fields.toString().stored()

In [25]:
!pip install sentencepiece
import sentencepiece
from transformers import BartTokenizer, BartForConditionalGeneration
SUMMARY_TOKENIZER = BartTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-6')
SUMMARY_MODEL = BartForConditionalGeneration.from_pretrained('sshleifer/distilbart-cnn-12-6')

#from transformers import PegasusTokenizer, PegasusForConditionalGeneration
#SUMMARY_TOKENIZER = PegasusTokenizer.from_pretrained('mayu0007/pegasus_large_covid')
#SUMMARY_MODEL = PegasusForConditionalGeneration.from_pretrained('mayu0007/pegasus_large_covid')

SUMMARY_MODEL.to(torch_device)
SUMMARY_MODEL.eval()
def Summarizer(string_all):    
    if 1==1:
        ## try generating an exacutive summary with bart abstractive summarizer
        allAnswersTxt = string_all.replace('\n','')

        answers_input_ids = SUMMARY_TOKENIZER.batch_encode_plus([allAnswersTxt], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
        #answers_input_ids = SUMMARY_TOKENIZER.prepare_seq2seq_batch([allAnswersTxt], return_tensors='pt', truncation=True, padding='longest')['input_ids'].to(torch_device)
        summary_ids = SUMMARY_MODEL.generate(answers_input_ids,
                                               num_beams=4,
                                               length_penalty=0.8,
                                               repetition_penalty=2.0,
                                               min_length=5,
                                               no_repeat_ngram_size=0,
                                                do_sample=False )

        exec_sum = SUMMARY_TOKENIZER.decode(summary_ids.squeeze(), skip_special_tokens=True)
        return exec_sum

# Program to find most frequent  
# element in a list 
  
from collections import Counter 
  
def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0]

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,AutoModelForSequenceClassification
#hg_model_hub_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/albert-xxlarge-v2-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/bart-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/electra-large-discriminator-snli_mnli_fever_anli_R1_R2_R3-nli"
hg_model_hub_name = "ynie/xlnet-large-cased-snli_mnli_fever_anli_R1_R2_R3-nli"

tokenizer_nli = AutoTokenizer.from_pretrained(hg_model_hub_name, use_fast=False)
model_nli = AutoModelForSequenceClassification.from_pretrained(hg_model_hub_name)
model_nli.to(torch_device)
model_nli.eval()

In [56]:
def CheckEntailmentNeturalorContradict(ranked_aswers,string_all,pandasData):
  if 1==1:
    premise_list=[]
    hypothesis_list=[]
    entailment_prob=[]
    neutral_prob=[]
    contradict_prob=[]
    final_result_list=[]
    if len(ranked_aswers)>8:
      v_range=8
    else:
      v_range=len(ranked_aswers)
    for i in range(v_range):
      if pandasData[i][2]>=0.4:
        max_length = 512
        candidate_answer=Summarizer(ranked_aswers[i])
        #premise = "symptoms of covid-19 range from none (asymptomatic) to severe pneumonia and it can be fatal. fever and cough are the most common symptoms in patients with 2019-ncov infection. Several people have muscle soreness or fatigue as well as ards. diarrhea, hemoptysis, headache, sore throat, shock, and other symptoms only occur in a small number of patients"
        if len(candidate_answer)>len(ranked_aswers[i]):
          premise = ranked_aswers[i]
        else:
          premise=candidate_answer
        #premise = "covid-19 symptom is highly various in each patient, with fever, fatigue, shortness of breath, and cough as the main presenting symptoms. patient with covid-19 may shows severe symptom with severe pneumonia and ards, mild symptom resembling simple upper respiration tract infection, or even completely asymptomatic. approximately 80 % of cases is mild "
        hypothesis= string_all 
        tokenized_input_seq_pair = tokenizer_nli.encode_plus(premise, hypothesis,
                                                      max_length=max_length,
                                                      return_token_type_ids=True, truncation=True)

        input_ids = torch.Tensor(tokenized_input_seq_pair['input_ids']).long().unsqueeze(0).to(torch_device)
        # remember bart doesn't have 'token_type_ids', remove the line below if you are using bart.
        token_type_ids = torch.Tensor(tokenized_input_seq_pair['token_type_ids']).long().unsqueeze(0).to(torch_device)
        attention_mask = torch.Tensor(tokenized_input_seq_pair['attention_mask']).long().unsqueeze(0).to(torch_device)

        outputs = model_nli(input_ids,
                      attention_mask=attention_mask,
                      token_type_ids=token_type_ids,
                      labels=None)
        # Note:
        # "id2label": {
        #     "0": "entailment",
        #     "1": "neutral",
        #     "2": "contradiction"
        # },

        predicted_probability = torch.softmax(outputs[0], dim=1)[0].tolist()  # batch_size only one

        # print("Premise:", premise)
        premise_list.append(premise)
        # print("Hypothesis:", hypothesis)
        hypothesis_list.append(hypothesis)
        # print("input Ans:",ranked_aswers[i])
        # print("Entailment:", predicted_probability[0])
        entailment_prob.append(predicted_probability[0])
        # print("Neutral:", predicted_probability[1])
        neutral_prob.append(predicted_probability[1])
        # print("Contradiction:", predicted_probability[2])
        contradict_prob.append(predicted_probability[2])
    if len(premise_list)>0:
      avg_entailment=np.average(entailment_prob)
      avg_neutral=np.average(neutral_prob)
      avg_contradiction=np.average(contradict_prob) 
      zipped_list = zip(entailment_prob,neutral_prob,contradict_prob)
      for key,val in enumerate(hypothesis_list):
        if neutral_prob[key]>0.8:
          final_result='Neutral'
        elif entailment_prob[key]>contradict_prob[key]:
          final_result='True'
          final_result_list.append(final_result)
        else:
          final_result='False'
          final_result_list.append(final_result)
      if len(final_result_list)>0:
          verdict=most_frequent(final_result_list)
      else :
          verdict='Neutral'
    else:
      zipped_list = zip(0,1,0)
      verdict='Neutral'
  return zipped_list,verdict

# Test

In [28]:
#!pip install pyserini==0.8.1.0
from pyserini.search import pysearch

import re
import sys
!pip install anglicize
from anglicize import anglicize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
!python -m pip install -U spacy
from spacy.tokenizer import Tokenizer
from nltk.tokenize import word_tokenize
from spacy.lang.en import English
import spacy
#!python -m spacy download en_core_web_sm
import spacy.cli # Colab fix
spacy.cli.download("en_core_web_sm") # Colab fix
import en_core_web_sm
nlp = en_core_web_sm.load()
nlp.add_pipe("merge_noun_chunks")
# nlp.add_pipe(nlp.create_pipe('merge_noun_chunks')) # Colab fix?
tokenizer = Tokenizer(nlp.vocab)
from timeit import default_timer as timer
import time
import json
import numpy as np
import statistics
import pandas as pd

DATA_TRAIN = "/content/train.jsonl" # Change this
DATA_DEV = "/content/dev.jsonl" # Change this

dataset_path = DATA_DEV
#dataset_path = DATA_TRAIN

with open(dataset_path, "r", encoding="utf-8") as f:
    dataset = [json.loads(line) for line in f]

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

Requirement already up-to-date: spacy in /usr/local/lib/python3.7/dist-packages (3.1.0)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [29]:
luceneDir = 'indexes/fever/lucene-index-fever-paragraph'
searcher = pysearch.SimpleSearcher(luceneDir)

def get_relevant_pages(claim, FULL_CLAIM, SPACY_ENTITIES, CASE_ENTITIES, ADD_SYNONYMS, ADD_SIMILAR, 
          LINKED_PAGES, LINKING_PAGES, CATCH_VERBS, TFIDF_TOP, FIND_ALL, N_RESULTS=70, SIMILAR_THRESHOLD=0.85, TFIDF_TOP_N=3):
    
    if LINKING_PAGES:
        import sqlalchemy as sqla

        db_fullpath = "D:/FEVER/feverous_wikiv1.db" # Change this
        db = sqla.create_engine("sqlite:///{}".format(db_fullpath))

        from sqlalchemy.orm import sessionmaker

        Session = sessionmaker(bind=db)
        session = Session()
    
    if not SPACY_ENTITIES and not CASE_ENTITIES and not LINKED_PAGES:
        FULL_CLAIM = True
    
    start_timer = timer()

    claim = nlp(claim.replace(" ", " ").replace("­", " ")) # Replaces the obnoxious space character with normal space

    keywords = set()
    entities = set()

    if SPACY_ENTITIES:
        spacy_entities = [entity.text for entity in claim.ents]
        entities.update(spacy_entities)

    if CASE_ENTITIES:
        case_entities = set()
        chunks = claim.noun_chunks
        for chunk in chunks:
            for token in tokenizer(chunk.text):
                if token.text[0].isupper():
                    case_entities.add(chunk.text)
                    break

        entities.update(case_entities)
        #print(case_entities)
        #print(entities)
        #sys.exit(0)

    keywords.update(entities)
    #print(keywords)

    if ADD_SYNONYMS:
        for token in claim:
            if token.is_stop:
                continue
            synonyms = wn.synsets(token.text)
            if synonyms and token:
                for synonym in synonyms:
                    if synonym.pos() == token.pos_[0].lower() and synonym.pos() == "n":
                        keywords.update([lemma.replace("_", " ") for lemma in synonym.lemma_names()])

    if ADD_SIMILAR:
        similar_check = [keyword for keyword in list(keywords) if keyword in vocab]

        if similar_check:
            similar_words = model.most_similar(positive=similar_check)
            for i in range(0, len(similar_words)):
                if i == 0 or similar_words[i][1] >= SIMILAR_THRESHOLD:
                    keywords.update([similar_words[i][0]])
    
    if CATCH_VERBS:
        for chunk in claim:
            if chunk.pos_ == "VERB" and chunk.dep_ != "case" and chunk.dep_ != "prep":
                keywords.update([chunk.lemma_])
                
    if TFIDF_TOP:
        tfidf_sorting = np.argsort(tfidf.transform([claim.text]).toarray()).flatten()[::-1]
        tfidf_len = len(tfidf_tokenize(claim.text))
        keywords.update(tfidf_features[tfidf_sorting][0:min([TFIDF_TOP_N, tfidf_len])])

    if not FULL_CLAIM:
        search_query = (", ".join(keywords) + '"' + '", "'.join(entities) + '"')
#            search_query = (", ".join(keywords) + ", ".join(entities))
    else:
        search_query = (claim.text + ' "' + '", "'.join(keywords) + '" "' + '", "'.join(entities) + '"')
#            search_query = (claim.text + " " + ", ".join(keywords) + ", ".join(entities))

    if FULL_CLAIM or keywords:
        try:
            lucene_hits = searcher.search(search_query.encode("utf-8"), k=N_RESULTS)
        except:
            lucene_hits = None
    else:
        lucene_hits = None

    hit_dictionary = {}

    if lucene_hits:

        for hit in lucene_hits:
            hit_dict = {"abstract": None, "real_abstract": None, "title": None}
            hit_dict['abstract']=hit.lucene_document.get("raw")
            hit_dict['real_abstract']=hit.lucene_document.get("raw")
            hit_dict['title'] = str(hit.docid)
            hit_dictionary[str(hit.docid)] = hit_dict
    
        linked_pages = set()
        if LINKED_PAGES:
            for hit in lucene_hits:
                links = re.findall(r"(?:\[\[)(.*?)(?:\|)", hit.raw)
                for link in links:
                    linked_pages.update([link.replace("_", " ").encode("latin-1", "ignore").decode("utf-8", "ignore")])
                    
        linking_pages = set()
        if LINKING_PAGES:
            linking_results = session.execute('select target, sources from inlinks where target IN ("{}")'.format('", "'.join([hit.docid.replace('"', '""') for hit in lucene_hits])))
            for row in linking_results:
                linking_pages.update(row["sources"].split(";"))
            """
            for hit in lucene_hits:
                linking_results = session.execute('select target, sources from inlinks where sources = "{}"'.format(hit.docid))
                for row in linking_results:
                    linking_pages.update(row["sources"].split(";"))"""

                    
        #print(lucene_hits[0].docid.encode("latin-1").decode("utf-8"))
        found_pages = [hit.docid for hit in lucene_hits]
        #found_pages = set([hit.docid.encode("latin-1", "ignore").decode("utf-8", "ignore") for page in lucene_hits])
        for i in range(0, len(found_pages)):

            try:
                found_pages[i] = found_pages[i].encode("latin-1", "ignore").decode("utf-8", "ignore")
                #print("Done:",found_pages[i])
            except:
                import regex
                print(found_pages[i], anglicize(found_pages[i]), regex.sub(r'[^\p{Latin}]', '', found_pages[i]).encode("latin-1").decode("utf-8"))
                
        found_pages = set(found_pages)

        hit_scores = [hit.score for hit in lucene_hits]
        hit_score_min = min(hit_scores)
        hit_score_25 = np.percentile(hit_scores, 25)
        hit_score_mean = np.mean(hit_scores)
        hit_score_median = statistics.median(hit_scores)
        hit_score_75 = np.percentile(hit_scores, 75)
        hit_score_max = max(hit_scores)
    
    else:
        
        is_found = False
        found_pages = set()
        entities = set()
        keywords = set()
        linked_pages = set()
        lucene_hits = []
        hit_scores = []
        hit_score_min = None
        hit_score_25 = None
        hit_score_mean = None
        hit_score_median = None
        hit_score_75 = None
        hit_score_max = None

    end_timer = timer()
    elapsed = int(end_timer - start_timer)
    elapsed_formatted = time.strftime("%H:%M:%S", time.gmtime(elapsed))
        
    result = [claim.text, elapsed_formatted, 
                found_pages, 
                keywords, 
                linked_pages, 
                linking_pages,
                len(lucene_hits), 
                hit_score_min, hit_score_25, hit_score_mean, hit_score_median, 
                hit_score_75, hit_score_max, hit_scores]

    #print(result)
    
    result = pd.DataFrame([result], columns=["CLAIM", "ELAPSED", 
                                             "PAGES_FOUND",
                                             "KEYWORDS", 
                                             "LINKED_PAGES",
                                             "LINKING_PAGES",
                                             "N_LUCENE_HITS", 
                                             "HIT_SCORE_MIN", "HIT_SCORE_25", "HIT_SCORE_MEAN", "HIT_SCORE_MEDIAN", 
                                             "HIT_SCORE_75", "HIT_SCORE_MAX", "HIT_SCORES"])
    
    return result, hit_dictionary

In [30]:
hit_stats, hit_dictionary = get_relevant_pages(dataset[0]["claim"], FULL_CLAIM=True, SPACY_ENTITIES=True, CASE_ENTITIES=True, ADD_SYNONYMS=False, ADD_SIMILAR=False, 
            LINKED_PAGES=False, LINKING_PAGES=False, CATCH_VERBS=False, TFIDF_TOP=False, FIND_ALL=True, N_RESULTS=70)

print([key for key in hit_dictionary])
hit_stats

['Ailanthus', 'Myrica', 'Polygonia c-aureum', 'Sasakia charonda', 'Scolitantides orion', 'Gravitcornutia sodalicia', 'Oegoconia caradjai', 'Gnathocolumna', 'Atrocenta', 'Rubroxena', 'Furcinetechma', 'Urania sloanus', 'Metacrias huttoni', 'Spiraea', 'Acacia sensu lato', 'Elysius amapaensis', 'Elysius anomala', 'Elysius barnesi', 'Elysius gladysia', 'Elysius systron', 'Metacrias erichrysa', 'Elysius atrobrunnea', 'Elysius jonesi', 'Elysius ochrota', 'Elysius pretiosa', 'Elysius subterra', 'Elysius carbonarius', 'Elysius chimaera', 'Elysius conjunctus', 'Elysius deceptura', 'Elysius flavoabdominalis', 'Elysius hades', 'Elysius lavinia', 'Elysius ordinaria', 'Elysius phantasma', 'Elysius terraoides', 'Elysius thrailkilli', 'Bonagota arizonae', 'Elysius ruffin', 'Izatha peroneanella', 'Elysius disciplaga', 'Elysius atrata', 'Elysius conspersus', 'Elysius melanoplaga', 'Apolychrosis schwerdtfegeri', 'Anopina guatemalana', 'Anopina phaeopina', 'Apolychrosis ambogonium', 'Atepa colaptes', 'Bic

,CLAIM,ELAPSED,PAGES_FOUND,KEYWORDS,LINKED_PAGES,LINKING_PAGES,N_LUCENE_HITS,HIT_SCORE_MIN,HIT_SCORE_25,HIT_SCORE_MEAN,HIT_SCORE_MEDIAN,HIT_SCORE_75,HIT_SCORE_MAX,HIT_SCORES
0,Tridrepana hainana is a moth found in China th...,00:00:00,"{Sasakia charonda, Elysius disciplaga, Ernocor...","{Lepidoptera, China, Tridrepana hainana, the D...",{},{},70,18.327576,18.327593,18.86494,18.505691,18.526397,25.760599,"[25.76059913635254, 22.830799102783203, 21.547..."


In [64]:
def get_answers(query, hit_dictionary):
    for idx,v in hit_dictionary.items():
        abs_dirty = v['abstract']
        real_abs_dirty = v['real_abstract']
        #abs_dirty = v['paragraph']
        # looks like the abstract value can be an empty list
        v['abstract_paragraphs'] = []
        v['abstract_full'] = ''
        v['real_abstract_full'] = ''
        v['real_abstract_paragraphs']=[]

        if abs_dirty:
            # looks like if it is a list, then the only entry is a dictionary wher text is in 'text' key
            # looks like it is broken up by paragraph if it is in that form.  lets make lists for every paragraph
            # and a new entry that is full abstract text as both could be valuable for BERT derrived QA

            if isinstance(abs_dirty, list):
                for p in abs_dirty:
                    v['abstract_paragraphs'].append(p['text'])
                    v['abstract_full'] += p['text'] + ' \n\n'

            # looks like in some cases the abstract can be straight up text so we can actually leave that alone
            if isinstance(abs_dirty, str):
                v['abstract_paragraphs'].append(abs_dirty)
                v['abstract_full'] += abs_dirty + ' \n\n'
        if real_abs_dirty:
            # looks like if it is a list, then the only entry is a dictionary wher text is in 'text' key
            # looks like it is broken up by paragraph if it is in that form.  lets make lists for every paragraph
            # and a new entry that is full abstract text as both could be valuable for BERT derrived QA


            if isinstance(real_abs_dirty, list):
                for p in real_abs_dirty:
                    v['real_abstract_paragraphs'].append(p['text'])
                    v['real_abstract_full'] += p['text'] + ' \n\n'

            # looks like in some cases the abstract can be straight up text so we can actually leave that alone
            if isinstance(abs_dirty, str):
                v['real_abstract_paragraphs'].append(real_abs_dirty)
                v['real_abstract_full'] += real_abs_dirty + ' \n\n'
        v["indexed_para"]=v["abstract_full"][len(v["real_abstract_full"]):]

    def embed_useT(module):
        with tf.Graph().as_default():
            sentences = tf.compat.v1.placeholder(tf.string)
            embed = hub.Module(module)
            embeddings = embed(sentences)
            session = tf.compat.v1.train.MonitoredSession()
        return lambda x: session.run(embeddings, {sentences: x})
    embed_fn = embed_useT('/root/kaggle/working/sentence_wise_email/module/module_useT')

    import numpy as np
    #Reconstruct text for generating the answer text from the BERT result, do the necessary text preprocessing and construct the answer.
    def reconstructText(tokens, start=0, stop=-1):
        tokens = tokens[start: stop]
        if '[SEP]' in tokens:
            sepind = tokens.index('[SEP]')
            tokens = tokens[sepind+1:]
        txt = ' '.join(tokens)
        txt = txt.replace(' ##', '')
        txt = txt.replace('##', '')
        txt = txt.strip()
        txt = " ".join(txt.split())
        txt = txt.replace(' .', '.')
        txt = txt.replace('( ', '(')
        txt = txt.replace(' )', ')')
        txt = txt.replace(' - ', '-')
        txt_list = txt.split(' , ')
        txt = ''
        nTxtL = len(txt_list)
        if nTxtL == 1:
            return txt_list[0]
        newList =[]
        for i,t in enumerate(txt_list):
            if i < nTxtL -1:
                if t[-1].isdigit() and txt_list[i+1][0].isdigit():
                    newList += [t,',']
                else:
                    newList += [t, ', ']
            else:
                newList += [t]
        return ''.join(newList)


    def makeBERTSQuADPrediction(document, question):
        ## we need to rewrite this function so that it chuncks the document into 250-300 word segments with
        ## 50 word overlaps on either end so that it can understand and check longer abstracts
        ## Get chuck of documents in order to overcome the BERT's max 512 token problem. First count the number of tokens. For example if it is
        ##   larger than 2048 token determine the split size and overlapping token count. Split the document into pieces to fed into BERT encoding. Overlapping
        ##      is necessary to maintain the coherence between the splits. If the token count is larger than some number 2560 token approximately this will fail.
        nWords = len(document.split())
        input_ids_all = QA_TOKENIZER.encode(question, document)
        tokens_all = QA_TOKENIZER.convert_ids_to_tokens(input_ids_all)
        overlapFac = 1.1
        if len(input_ids_all)*overlapFac > 2560:
            nSearchWords = int(np.ceil(nWords/6))
            fifth = int(np.ceil(nWords/5))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                        ' '.join(docSplit[fifth-int(nSearchWords*overlapFac/2):fifth+int(fifth*overlapFac/2)]),
                        ' '.join(docSplit[fifth*2-int(nSearchWords*overlapFac/2):fifth*2+int(fifth*overlapFac/2)]),
                        ' '.join(docSplit[fifth*3-int(nSearchWords*overlapFac/2):fifth*3+int(fifth*overlapFac/2)]),
                        ' '.join(docSplit[fifth*4-int(nSearchWords*overlapFac/2):fifth*4+int(fifth*overlapFac/2)]),
                        ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]

        elif len(input_ids_all)*overlapFac > 2048:
            nSearchWords = int(np.ceil(nWords/5))
            quarter = int(np.ceil(nWords/4))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                        ' '.join(docSplit[quarter-int(nSearchWords*overlapFac/2):quarter+int(quarter*overlapFac/2)]),
                        ' '.join(docSplit[quarter*2-int(nSearchWords*overlapFac/2):quarter*2+int(quarter*overlapFac/2)]),
                        ' '.join(docSplit[quarter*3-int(nSearchWords*overlapFac/2):quarter*3+int(quarter*overlapFac/2)]),
                        ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]        
            
        elif len(input_ids_all)*overlapFac > 1536:
            nSearchWords = int(np.ceil(nWords/4))
            third = int(np.ceil(nWords/3))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                        ' '.join(docSplit[third-int(nSearchWords*overlapFac/2):third+int(nSearchWords*overlapFac/2)]),
                        ' '.join(docSplit[third*2-int(nSearchWords*overlapFac/2):third*2+int(nSearchWords*overlapFac/2)]),
                        ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]        
            
        elif len(input_ids_all)*overlapFac > 1024:
            nSearchWords = int(np.ceil(nWords/3))
            middle = int(np.ceil(nWords/2))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                        ' '.join(docSplit[middle-int(nSearchWords*overlapFac/2):middle+int(nSearchWords*overlapFac/2)]),
                        ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
        elif len(input_ids_all)*overlapFac > 512:
            nSearchWords = int(np.ceil(nWords/2))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
        else:
            input_ids = [input_ids_all]
        absTooLong = False    
        
        answers = []
        cons = []
        #print(input_ids)
        for iptIds in input_ids:
            tokens = QA_TOKENIZER.convert_ids_to_tokens(iptIds)
            #print(tokens)
            sep_index = iptIds.index(QA_TOKENIZER.sep_token_id)
            num_seg_a = sep_index + 1
            num_seg_b = len(iptIds) - num_seg_a
            segment_ids = [0]*num_seg_a + [1]*num_seg_b
            assert len(segment_ids) == len(iptIds)
            n_ids = len(segment_ids)
            #print(n_ids)
            if n_ids < 512:
                outputs = QA_MODEL(torch.tensor([iptIds]).to(torch_device), 
                                        token_type_ids=torch.tensor([segment_ids]).to(torch_device))
                #print(QA_MODEL(torch.tensor([iptIds]).to(torch_device), 
                #                        token_type_ids=torch.tensor([segment_ids]).to(torch_device)))
            else:
                #this cuts off the text if its more than 512 words so it fits in model space
                #need run multiple inferences for longer text. add to the todo. I will try to expand this to longer sequences if it is necessary. 
                print('****** warning only considering first 512 tokens, document is '+str(nWords)+' words long.  There are '+str(n_ids)+ ' tokens')
                absTooLong = True
                outputs= QA_MODEL(torch.tensor([iptIds[:512]]).to(torch_device), 
                                        token_type_ids=torch.tensor([segment_ids[:512]]).to(torch_device))
            start_scores=outputs.start_logits
            end_scores=outputs.end_logits
            start_scores = start_scores[:,1:-1]
            end_scores = end_scores[:,1:-1]
            answer_start = torch.argmax(start_scores)
            answer_end = torch.argmax(end_scores)
            #print(answer_start, answer_end)
            answer = reconstructText(tokens, answer_start, answer_end+2)
        
            if answer.startswith('. ') or answer.startswith(', '):
                answer = answer[2:]
                
            c = start_scores[0,answer_start].item()+end_scores[0,answer_end].item()
            answers.append(answer)
            cons.append(c)
        
        maxC = max(cons)
        iMaxC = [i for i, j in enumerate(cons) if j == maxC][0]
        confidence = cons[iMaxC]
        answer = answers[iMaxC]
        
        sep_index = tokens_all.index('[SEP]')
        full_txt_tokens = tokens_all[sep_index+1:]
        
        abs_returned = reconstructText(full_txt_tokens)

        ans={}
        ans['answer'] = answer
        #print(answer)
        if answer.startswith('[CLS]') or answer_end.item() < sep_index or answer.endswith('[SEP]'):
            ans['confidence'] = -1000000
        else:
            #confidence = torch.max(start_scores) + torch.max(end_scores)
            #confidence = np.log(confidence.item())
            ans['confidence'] = confidence
        #ans['start'] = answer_start.item()
        #ans['end'] = answer_end.item()
        ans['abstract_bert'] = abs_returned
        ans['abs_too_long'] = absTooLong
        return ans

    from tqdm import tqdm
    def searchAbstracts(hit_dictionary, question):
        abstractResults = {}
        for k,v in tqdm(hit_dictionary.items()):
            abstract = v['abstract_full']
            indexed_para=v['indexed_para']
            if abstract:
                ans = makeBERTSQuADPrediction(abstract, question)
                if ans['answer']:
                    confidence = ans['confidence']
                    abstractResults[confidence]={}
                    abstractResults[confidence]['answer'] = ans['answer']
                    #abstractResults[confidence]['start'] = ans['start']
                    #abstractResults[confidence]['end'] = ans['end']
                    abstractResults[confidence]['abstract_bert'] = ans['abstract_bert']
                    abstractResults[confidence]['idx'] = k
                    abstractResults[confidence]['abs_too_long'] = ans['abs_too_long']

                    
        cList = list(abstractResults.keys())

        if cList:
            maxScore = max(cList)
            total = 0.0
            exp_scores = []
            for c in cList:
                s = np.exp(c-maxScore)
                exp_scores.append(s)
            total = sum(exp_scores)
            for i,c in enumerate(cList):
                abstractResults[exp_scores[i]/total] = abstractResults.pop(c)
        return abstractResults

    answers = searchAbstracts(hit_dictionary, query)
    
    FIND_PDFS=False 
    SEARCH_MEDRXIV=False

    workingPath = '/root/kaggle/working'
    import pandas as pd
    import re
    if FIND_PDFS:
        from metapub import UrlReverse
        from metapub import FindIt
    from IPython.core.display import display, HTML

    #from summarizer import Summarizer
    #summarizerModel = Summarizer()
    def displayResults(hit_dictionary, answers, question):
        
        question_HTML = '<div style="font-family: Times New Roman; font-size: 28px; padding-bottom:28px"><b>Query</b>: '+question+'</div>'
        #all_HTML_txt = question_HTML
        confidence = list(answers.keys())
        confidence.sort(reverse=True)
        
        confidence = list(answers.keys())
        confidence.sort(reverse=True)
        

        for c in confidence:
            if c>0 and c <= 1 and len(answers[c]['answer']) != 0:
                if 'idx' not in  answers[c]:
                    continue
                rowData = []
                idx = answers[c]['idx']
                title = hit_dictionary[idx]['title']

                
                full_abs = answers[c]['abstract_bert']
                bert_ans = answers[c]['answer']
                
                
                #split_abs = full_abs.split(bert_ans)
                #x=(split_abs[0][:split_abs[0].rfind('. ')+1])
                #sentance_beginning = x[x.rfind('. ')+1:] + split_abs[0][split_abs[0].rfind('. ')+1:]
                #sentance_beginning = split_abs[0][split_abs[0].rfind('.')+1:]
                x=''
                y=''
                z=''
                t=''
                regex = r"(\.\s[^0-9])(?!.*(\.\s[^0-9]))"
                split_abs = full_abs.split(bert_ans)
                matches = re.finditer(regex, split_abs[0], re.MULTILINE)
                for matchNum1, match in enumerate(matches, start=1):
                    y=(split_abs[0][:match.start()+1])
                matches = re.finditer(regex, y, re.MULTILINE)
                for matchNum2, match in enumerate(matches, start=1):
                    x=(y[match.start()+1:])
                if x=='':
                    x=split_abs[0]
                matches = re.finditer(regex, split_abs[0], re.MULTILINE)
                for matchNum3, match in enumerate(matches, start=1):
                    z=(split_abs[0][match.start()+1:])  
                sentance_beginning = x+z
                regex2=r"(.*?(?<!\b\w)[.?!])\s+[a-zA-Z0-9]"
                if len(split_abs) == 1:
                    sentance_end_pos = len(full_abs)
                    sentance_end =''
                else:
                    matches = re.finditer(regex2, split_abs[1], re.MULTILINE)
                    for matchNum4, match in enumerate(matches, start=1):
                        if matchNum4==1:
                            t=(split_abs[1][:match.end()-2])
                    sentance_end_pos = split_abs[1].find('. ')+1
                    if sentance_end_pos == 0:
                        sentance_end = split_abs[1]
                    else:
                        sentance_end = t
                #if len(split_abs) == 1:
                #    sentance_end_pos = len(full_abs)
                #    sentance_end =''
                #else:
                #    sentance_end_pos = split_abs[1].find('. ')+1
                #    if sentance_end_pos == 0:
                #      sentance_end = split_abs[1]
                #    else:
                #      sentance_end = split_abs[1][:sentance_end_pos]
                    
                #sentance_full = sentance_beginning + bert_ans+ sentance_end
                answers[c]['full_answer'] = sentance_beginning+bert_ans+sentance_end
                answers[c]['partial_answer'] = bert_ans+sentance_end
                answers[c]['sentence_beginning'] = sentance_beginning
                answers[c]['sentence_end'] = sentance_end
                answers[c]['title'] = title
            else:
                answers.pop(c)
        
        
        ## now rerank based on semantic similarity of the answers to the question
        ## Universal sentence encoder
        cList = list(answers.keys())
        allAnswers = [answers[c]['full_answer'] for c in cList]
        
        messages = [question]+allAnswers
        
        encoding_matrix = embed_fn(messages)
        similarity_matrix = np.inner(encoding_matrix, encoding_matrix)
        rankings = similarity_matrix[1:,0]
        
        for i,c in enumerate(cList):
            answers[rankings[i]] = answers.pop(c)

        ## now form pandas dv
        confidence = list(answers.keys())
        confidence.sort(reverse=True)
        pandasData = []
        ranked_aswers = []
        full_abs_list=[]
        for c in confidence:
            rowData=[]
            title = answers[c]['title']
            idx = answers[c]['idx']
            rowData += [idx]            
            sentance_html = '<div>' +answers[c]['sentence_beginning'] + " <font color='red'>"+answers[c]['answer']+"</font> "+answers[c]['sentence_end']+'</div>'
            answer_key = answers[c]['answer'].split("\t")[-1].strip() if "\t" in answers[c]['answer'] else answers[c]['answer'].strip()

            # rowData += [sentance_html, answer_key, c,title]
            rowData += [sentance_html, c,title]
            pandasData.append(rowData)
            ranked_aswers.append(' '.join([answers[c]['full_answer']]))
            full_abs_list.append(' '.join([answers[c]['abstract_bert']]))
        
        if FIND_PDFS or SEARCH_MEDRXIV:
            pdata2 = []
            pm_ids = []
            for rowData in pandasData:
                rd = rowData
                idx = rowData[0]
        else:
            pdata2 = pandasData
            
        
        display(HTML(question_HTML))

        if USE_SUMMARY:
            ## try generating an exacutive summary with bart abstractive summarizer
            allAnswersTxt = ' '.join(ranked_aswers[:10]).replace('\n','')
        #    exec_sum = summarizerModel(allAnswersTxt, min_length=1, max_length=500)    
        #   execSum_HTML = '<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:18px"><b>BERT Extractive Summary:</b>: '+exec_sum+'</div>'

            answers_input_ids = SUMMARY_TOKENIZER.batch_encode_plus([allAnswersTxt], return_tensors='pt', max_length=2048)['input_ids'].to(torch_device)
            #answers_input_ids = SUMMARY_TOKENIZER.prepare_seq2seq_batch([allAnswersTxt], return_tensors='pt', truncation=True, padding='longest')['input_ids'].to(torch_device)
            summary_ids = SUMMARY_MODEL.generate(answers_input_ids,
                                                num_beams=10,
                                                length_penalty=1.1,
                                                max_length=2048,
                                                min_length=64,
                                                no_repeat_ngram_size=0,
                                                    do_sample=False )

            exec_sum = SUMMARY_TOKENIZER.decode(summary_ids.squeeze(), skip_special_tokens=True)
            execSum_HTML = '<div style="font-family: Times New Roman; font-size: 18px; margin-bottom:1pt"><b>BART Abstractive Summary:</b>: '+exec_sum+'</div>'
            display(HTML(execSum_HTML))
            warning_HTML = '<div style="font-family: Times New Roman; font-size: 12px; padding-bottom:12px; color:#CCCC00; margin-top:1pt"> Warning this is an autogenerated summary based on semantic search of abstracts, always examine the sources before accepting this conclusion.  If the evidence only mentions topic in passing or the evidence is not clear, the summary will likely not clearly answer the question.</div>'
            display(HTML(warning_HTML))

    #    display(HTML('<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:18px"><b>Body of Evidence:</b></div>'))
        
        if FIND_PDFS or SEARCH_MEDRXIV:
            # df = pd.DataFrame(pdata2, columns = ['Lucene ID', 'BERT-SQuAD Answer with Highlights', 'Answer Key', 'Confidence', 'Title/Link'])
            df = pd.DataFrame(pdata2, columns = ['Lucene ID', 'BERT-SQuAD Answer with Highlights', 'Confidence', 'Title/Link'])
        else:
            # df = pd.DataFrame(pdata2, columns = ['Lucene ID', 'BERT-SQuAD Answer with Highlights', 'Answer Key', 'Confidence', 'Title/Link'])
            df = pd.DataFrame(pdata2, columns = ['Lucene ID', 'BERT-SQuAD Answer with Highlights', 'Confidence', 'Title/Link'])
            
        display(HTML(df.to_html(render_links=True, escape=False)))
        return full_abs_list,ranked_aswers,pandasData

    return displayResults(hit_dictionary, answers, query)




In [78]:
for data in dataset[:1]:
    claim = data["claim"]
    hit_stats, hit_dictionary = get_relevant_pages(claim, FULL_CLAIM=True, SPACY_ENTITIES=True, CASE_ENTITIES=True, ADD_SYNONYMS=False, ADD_SIMILAR=False, 
                            LINKED_PAGES=False, LINKING_PAGES=False, CATCH_VERBS=False, TFIDF_TOP=False, FIND_ALL=True, N_RESULTS=70)
    
    full_abs_list, ranked_aswers, pandasData = get_answers(claim, hit_dictionary)

    # counter = 0
    # for data in pandasData:
    #     if data[2] > 0.4:
    #         counter += 1
    # print(counter)

    evidence_probs, verdict = CheckEntailmentNeturalorContradict(ranked_aswers,claim,pandasData)
    evidence_probs = list(a)

    # print(len(list(evidence_probs)))
    # print(verdict)

    if verdict == "False":
        predicted_label = "REFUTES"
    elif verdict == "True":
        predicted_label = "SUPPORTS"
    else:
        predicted_label = "NOT ENOUGH INFO"
    
    output = {"id": "", "predicted_label": predicted_label, "predicted_evidence": [[data[3], data[1], ""] for data in pandasData[:min([len(pandasData), 5])]]}

    print(output)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


  1%|▏         | 1/70 [00:00<00:13,  5.25it/s]

****** warning only considering first 512 tokens, document is 351 words long.  There are 571 tokens


  3%|▎         | 2/70 [00:00<00:13,  4.95it/s]

****** warning only considering first 512 tokens, document is 585 words long.  There are 618 tokens


...

 33%|███▎      | 23/70 [00:02<00:04, 10.39it/s]

****** warning only considering first 512 tokens, document is 441 words long.  There are 537 tokens


100%|██████████| 70/70 [00:04<00:00, 15.55it/s]


,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link
0,Elysius pretiosa,pretiosa | [ [ binomial _ nomenclature | binomial name ] ] sentence _ 0 elysius pretiosa is a [ [ moth | moth ] ] of the family [ [ erebidae | erebidae ] ] [ [ species _ description | first described ] ] by [ [ peter _ jorgensen | peter jorgensen ] ] in 1935. sentence _ 1 it is found in paraguay .,0.855801,Elysius pretiosa
1,Dorithia wellingana,wellingana | [ [ binomial _ nomenclature | binomial name ] ] sentence _ 0 dorithia wellingana is a species of [ [ moth | moth ] ] of the family [ [ tortricidae | tortricidae ] ]. sentence _ 1 it is found in guatemala .,0.844346,Dorithia wellingana
2,Dorithia paraviridana,paraviridana | [ [ binomial _ nomenclature | binomial name ] ] sentence _ 0 dorithia paraviridana is a species of [ [ moth | moth ] ] of the family [ [ tortricidae | tortricidae ] ]. sentence _ 1 it is found in guatemala .,0.824886,Dorithia paraviridana
3,Scolitantides orion,"the butterflies in the early spring (emerging in a warm room already in february according to tumma), and in the south again in august, very local, being absent from large districts, but generally not rare, particularly on chalk in stony places. in china, [ [ korea | korea ] ] and amurland the species is much more generally distributed than in europe, but always confined to the (rocky) localities of the food plant.",0.712812,Scolitantides orion
4,Elysius ochrota,table _ 0 elysius ochrota | [ [ taxonomy _ (biology) | scientific classification ] ] [ [ elysius ] ] | kingdom : [ [ animal | animalia ] ] | phylum : [ [ arthropod | arthropoda ] ] | class : [ [ insect | insecta ] ] | order : [ [ lepidoptera | lepidoptera ] ] | superfamily : [ [ noctuoidea | noctuoidea ] ] | family : [ [ erebidae | erebidae ] ] | subfamily : [ [ arctiinae | arctiinae ] ] | genus : [ [ elysius _ (moth) | elysius ] ] | species : e. ochrota | [ [ binomial _ nomenclature | binomial name ] ] sentence _ 0 elysius ochrota is a [ [ moth | moth ] ] of the family [ [ erebidae | erebidae ] ] [ [ species _ description | first described ] ] by [ [ george _ hampson | george hampson ] ] in 1901.,0.710083,Elysius ochrota
5,Elysius lavinia,table _ 0 elysius lavinia | [ [ taxonomy _ (biology) | scientific classification ] ] [ [ elysius ] ] | kingdom : [ [ animal | animalia ] ] | phylum : [ [ arthropod | arthropoda ] ] | class : [ [ insect | insecta ] ] | order : [ [ lepidoptera | lepidoptera ] ] | superfamily : [ [ noctuoidea | noctuoidea ] ] | family : [ [ erebidae | erebidae ] ] | subfamily : [ [ arctiinae | arctiinae ] ] | genus : [ [ elysius _ (moth) | elysius ] ] | species : e. lavinia | [ [ binomial _ nomenclature | binomial name ] ] | [ [ synonym _ (taxonomy) | synonyms ] ] sentence _ 0 elysius lavinia is a [ [ moth | moth ] ] of the family [ [ erebidae | erebidae ] ] [ [ species _ description | first described ] ] by [ [ herbert _ druce | herbert druce ] ] in 1906.,0.705732,Elysius lavinia
6,Elysius melanoplaga,table _ 0 elysius melanoplaga | [ [ taxonomy _ (biology) | scientific classification ] ] [ [ elysius ] ] | kingdom : [ [ animal | animalia ] ] | phylum : [ [ arthropod | arthropoda ] ] | class : [ [ insect | insecta ] ] | order : [ [ lepidoptera | lepidoptera ] ] | superfamily : [ [ noctuoidea | noctuoidea ] ] | family : [ [ erebidae | erebidae ] ] | subfamily : [ [ arctiinae | arctiinae ] ] | genus : [ [ elysius _ (moth) | elysius ] ] | species : e. melanoplaga | [ [ binomial _ nomenclature | binomial name ] ] | [ [ synonym _ (taxonomy) | synonyms ] ] sentence _ 0 elysius melanoplaga is a [ [ moth | moth ] ] of the family [ [ erebidae | erebidae ] ] [ [ species _ description | first described ] ] by [ [ george _ hampson | george hampson ] ] in 1901.,0.695277,Elysius melanoplaga
7,Elysius chimaera,table _ 0 elysius chimaera | [ [ taxonomy _ (biology) | scientific classification ] ] [ [ elysius ] ] | kingdom : [ [ animal | animalia ] ] | phylum : [ [ arthropod | arthropoda ] ] | class : [ [ in

{'id': '', 'predicted_label': 'REFUTES', 'predicted_evidence': [['Elysius pretiosa', "<div> pretiosa | [ [ binomial _ nomenclature | binomial name ] ] sentence _ 0 elysius pretiosa is a [ [ moth | moth ] ] of the family [ [ erebidae | erebidae ] ] [ [ species _ description | first described ] ] by [ [ peter _ jorgensen | peter jorgensen ] ] in 1935. sentence _ 1 it is found  <font color='red'>in paraguay</font> .</div>", ''], ['Dorithia wellingana', "<div> wellingana | [ [ binomial _ nomenclature | binomial name ] ] sentence _ 0 dorithia wellingana is a species of [ [ moth | moth ] ] of the family [ [ tortricidae | tortricidae ] ]. sentence _ 1 it is found  <font color='red'>in guatemala</font> .</div>", ''], ['Dorithia paraviridana', "<div> paraviridana | [ [ binomial _ nomenclature | binomial name ] ] sentence _ 0 dorithia paraviridana is a species of [ [ moth | moth ] ] of the family [ [ tortricidae | tortricidae ] ]. sentence _ 1 it is found  <font color='red'>in guatemala</font> .